In [1121]:
import json
from konlpy.tag import Kkma
from konlpy.utils import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC,SVR
from sklearn.metrics import accuracy_score,mean_squared_error,roc_auc_score,f1_score
import numpy as np
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn import grid_search, cross_validation

C:\Users\kvtor\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\kvtor\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [872]:
REVIEWS = "cs_reviews"
POS = "pos"

In [873]:
f = open("../reviews_in_one_txt/" + REVIEWS + ".txt","r",encoding="utf-8").readlines()

In [874]:
g = open("../reviews_in_one_txt/" + REVIEWS + "_" + POS + ".txt","r",encoding="utf-8").readlines()

In [875]:
# kkma = Kkma()
# with open("../reviews_in_one_txt/" + REVIEx`WS+"_nouns.txt","w",encoding="utf-8") as out:
#     for line in f:
#         course = json.loads(line)
#         nouns = kkma.nouns(course["review_text"])
# #         print(nouns)
#         out.write(" ".join(nouns)+"\n")
        

In [1058]:

# mp = {"A":5,"B":4,"C":3,"D":2,"F":1}
mp = {"A":1,"B":1,"C":0,"D":0,"F":0}

In [1059]:
count = {}
for i in range(len(f)):
    course = json.loads(f[i])
#     print(course)
    if course["강의"] != "?" and len(g[i])>1:
        if mp[course["강의"]] not in count:
            count[mp[course["강의"]]] = 0
        count[mp[course["강의"]]] += 1
        
print(count)
upper_limit = min(count.values())
upper_limit *= 2 
print(upper_limit)

{0: 138, 1: 364}
276


In [1060]:
concat_nouns_list = []
for k,v in count.items():
    count[k] = 0

labels = []
important_pos = ["NNG","MAG","VV","OL","VA"]
for i in range(len(f)):
    line = f[i]
    course = json.loads(line)
    if course["강의"] == "?" or len(g[i])<=1:
        continue
    pos_list = g[i].split()
    l = [x[1:-1].split(',') for x in pos_list]
#     print(l[:2])
    to_be_joined = []
    for p in l:
        if p[1] in important_pos:
            to_be_joined.append(p[0])
    concat_nouns = " ".join(to_be_joined)
    if count[mp[course["강의"]]] < upper_limit:
        concat_nouns_list.append(concat_nouns)
        labels.append(int(mp[course["강의"]]))
        count[mp[course["강의"]]] += 1

In [1085]:
labels = np.array(labels)
concat_nouns_list = np.array(concat_nouns_list)
data_train, data_test, y_train, y_test = train_test_split(concat_nouns_list,
                                    labels,test_size=0.1,random_state=42)

In [1086]:
len(data_train)

372

In [1087]:
len(y_train)

372

In [1088]:
list(data_test)[3]

'수업 안 듣 혼자 공부 더 좋 수업 전달 자료 대충 만들 느낌 빠지 내용 많 짜증나 조교 있 채점 기준 마음대로 바꾸 틀리 이유 대하 설명 논리 없'

In [1089]:
y_train[2]

1

In [1090]:
def parse_long_str(s):
    s = s.split(',')
    s = [x.strip() for x in s]
    return s
good_lecture = parse_long_str("사랑, 천사, 친절, 유명, 짱, 참여, 최고, 만족, 유익, 열정적, 갓, 답변, 명강의")
bad_lecture = parse_long_str("집중, 혼자, 부족, 자습, 당황, 최악, 인생, 불편, 복잡, 불만, 악명, 비추, 난이(도)")
#todo try weighted average of w2v
scores = {}
def add_to_dict(l,score):
    global scores
    print("Added " + str(len(l)))
    for i in range(len(l)):
        scores[l[i]] = score
add_to_dict(good_lecture,1.0)
add_to_dict(bad_lecture,-1.0)

Added 13
Added 13


In [1091]:
def enlarge_train_data(data_train,y_train):
    x = list(data_train)
    y = list(y_train)
    for k,v in scores.items():
#         x.append(str(k)+"\n")
#         unit = "(%s,NNG) " % (str(k))
        unit = "%s " % (str(k))
        x.append(unit*1)
        
        if v > 0:
            y.append(int(mp['A']))
        else:
            y.append(int(mp['F']))
    
    return np.array(x),np.array(y)

# data_train,y_train = enlarge_train_data(data_train,y_train)

In [1092]:
len(data_train)

372

In [1093]:
len(y_train)

372

In [1094]:
vectorizer = TfidfVectorizer(min_df=3, max_df = 0.8,max_features=200000)

In [1095]:
# vectirizer.fit(..)
# X_train = vectorizer.transform(data_train)
X_train = vectorizer.fit_transform(data_train)
X_test = vectorizer.transform(data_test)

In [1096]:
tfidf_features = vectorizer.get_feature_names()

In [1097]:
len(tfidf_features)

917

In [1098]:
tfidf_features

['assignment',
 'board',
 'code',
 'cpu',
 'cs',
 'design',
 'document',
 'dp',
 'ds',
 'free',
 'gdb',
 'hci',
 'homework',
 'hour',
 'java',
 'kens',
 'klms',
 'lab',
 'memory',
 'np',
 'office',
 'os',
 'otl',
 'pa',
 'pintos',
 'pl',
 'post',
 'pp',
 'ppt',
 'pre',
 'printf',
 'project',
 'python',
 'racket',
 'reading',
 'scala',
 'scheme',
 'se',
 'search',
 'sort',
 'sp',
 'sql',
 'stanford',
 'ta',
 'vhdl',
 '가까이',
 '가깝',
 '가끔',
 '가능',
 '가도',
 '가량',
 '가르',
 '가르치',
 '가면',
 '가보',
 '가시',
 '가시길',
 '가을',
 '가장',
 '가지',
 '가치',
 '각각',
 '각종',
 '갈리',
 '갈수록',
 '감사',
 '감안',
 '감점',
 '갑자기',
 '강의',
 '강제',
 '강추',
 '같이',
 '개념',
 '개론',
 '개발',
 '개선',
 '개요',
 '개인',
 '개인적',
 '거리',
 '거의',
 '거지',
 '걱정',
 '건의',
 '건지',
 '걸리',
 '검색',
 '게다',
 '게임',
 '겨우',
 '결과',
 '결국',
 '결론',
 '결석',
 '결정',
 '겹치',
 '경우',
 '경향',
 '경험',
 '계속',
 '계시',
 '계열',
 '계획서',
 '고등',
 '고민',
 '고사',
 '고생',
 '고통',
 '공개',
 '공부',
 '공유',
 '공지',
 '공학',
 '과목',
 '과정',
 '과제',
 '관련',
 '관리',
 '관심',
 '관하',
 '광무',
 '괜찮',
 '괴수',
 '굉장히',
 '교과서',
 '교수'

In [1099]:
print(X_train.shape)

(372, 917)


In [1100]:
print(X_test.shape)

(42, 917)


In [1115]:
clf = LinearSVC()
clf.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [1116]:
predictions = clf.predict(X_test)
accuracy_score(y_test,predictions)

0.8095238095238095

In [1117]:
mean_squared_error(y_test,predictions)

0.19047619047619047

In [1118]:
predictions

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [1119]:
y_test

array([1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1])

In [1120]:
for i in range(len(predictions)):
    if predictions[i]!= y_test[i]:
#         print("pred",predictions[i],"real",y_test[i])
#         print(data_test[i])
        pass
        

In [1114]:
micro = f1_score(y_test, predictions, average='micro')  
macro = f1_score(y_test, predictions, average='macro')  
print(micro,macro)
f1_score(y_test, predictions, average='weighted')  

0.7380952380952381 0.49945828819068255


0.6475777743383377

In [1122]:
roc_auc_score(y_test,predictions)

0.6666666666666667

In [1131]:
LinearSVC().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

In [1138]:
params = {'penalty':['l2'],
          'C': [10**i for i in range(-2,2)]}

gs2 = grid_search.GridSearchCV(
    estimator=LinearSVC(),
    param_grid=params,
    cv=5,
    scoring='f1'
)

#fit the grid search object to our new dataset
print ('Fitting grid search...')
gs2.fit(X_train, y_train)
print ("Grid search fitted.")

Fitting grid search...
Grid search fitted.


In [1139]:
print (gs2.best_estimator_)
gs2.grid_scores_

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


[mean: 0.79612, std: 0.00124, params: {'C': 0.01, 'penalty': 'l2'},
 mean: 0.80401, std: 0.00920, params: {'C': 0.1, 'penalty': 'l2'},
 mean: 0.76329, std: 0.02554, params: {'C': 1, 'penalty': 'l2'},
 mean: 0.74119, std: 0.03044, params: {'C': 10, 'penalty': 'l2'}]